# Import Basic libraries

In [18]:
import io
import requests
import pandas as pd
import numpy as np

# Download packages for maps

In [19]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



# create the dataframe reading the given input file

In [29]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header =0, flavor='bs4', )[0]
dfs.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [30]:
dfs2 = dfs[dfs['Borough']!= 'Not assigned']
dfs2.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park

In [31]:
dfs2['UpdatedNeighbourhood'] = np.where(dfs2.Neighbourhood == 'Not assigned',dfs2.Borough,dfs2.Neighbourhood)
dfs2.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postcode,Borough,Neighbourhood,UpdatedNeighbourhood
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M5A,Downtown Toronto,Regent Park,Regent Park
6,M6A,North York,Lawrence Heights,Lawrence Heights
7,M6A,North York,Lawrence Manor,Lawrence Manor
8,M7A,Queen's Park,Not assigned,Queen's Park
10,M9A,Etobicoke,Islington Avenue,Islington Avenue
11,M1B,Scarborough,Rouge,Rouge
12,M1B,Scarborough,Malvern,Malvern


# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [32]:
dfs2.groupby(['Postcode','Borough'], as_index=False).agg([('UpdatedNeighbourhood', ', '.join)])
dfs2.reset_index(level=0, inplace=True) 
dfs2.head()

,index,Postcode,Borough,Neighbourhood,UpdatedNeighbourhood
0,2,M3A,North York,Parkwoods,Parkwoods
1,3,M4A,North York,Victoria Village,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park,Regent Park
4,6,M6A,North York,Lawrence Heights,Lawrence Heights


# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [33]:
dfs2.shape

(211, 5)

# 2nd part of the assignment - Reading GEO DATA

In [34]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
# pd.merge(restaurant_ids_dataframe, restaurant_review_frame, on='business_id', how='outer')
newdf = pd.merge( geo_data, dfs2, how='left', right_on = 'Postcode', left_on = 'Postal Code')
newdf.drop(['index','Postal Code','Neighbourhood'], axis=1, inplace=True) 
newdf.rename(columns={'UpdatedNeighbourhood':'Neighbourhood'}, inplace=True)
neworder = ['Postcode','Borough','Neighbourhood','Latitude','Longitude']
newdf=newdf.reindex(columns=neworder)
newdf.head(10)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 3rd part of the assignement - Explore and cluster the neighborhoods in Toronto.

In [36]:
Torontodf = newdf[newdf['Borough'].str.contains("Toronto")]
Torontodf.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
67,M4E,East Toronto,The Beaches,43.676357,-79.293031
71,M4K,East Toronto,The Danforth West,43.679557,-79.352188
72,M4K,East Toronto,Riverdale,43.679557,-79.352188
73,M4L,East Toronto,The Beaches West,43.668999,-79.315572
74,M4L,East Toronto,India Bazaar,43.668999,-79.315572
75,M4M,East Toronto,Studio District,43.659526,-79.340923
76,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
77,M4P,Central Toronto,Davisville North,43.712751,-79.390197
78,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
79,M4S,Central Toronto,Davisville,43.704324,-79.388790


In [37]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [40]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(Torontodf['Latitude'], Torontodf['Longitude'], Torontodf['Borough'], Torontodf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto